In [ ]:
from keras_contrib.layers import CRF
# import keras_contrib.layers

In [2]:
from tensorflow.keras.models import Model

AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

In [4]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import gensim

train_data = 'data/ctb5.1-pos/ctb5.1-pos/train.tsv'
test_data = 'data/ctb5.1-pos/ctb5.1-pos/test.tsv'

In [5]:
# pip install protobuf==3.20 -us

## 数据读取

In [6]:
def get_data(file_path):
    data = pd.read_csv(file_path, sep='\t', skip_blank_lines=False, header=None)
    # 取出文本部分
    content = data[0]
    # 取出标签部分
    label = data[1]
    
    return content, label

In [7]:
# 读取数据集
X_train, y_train = get_data(train_data)
X_test, y_test = get_data(test_data)

## 数据预处理与格式转化

In [8]:
# 构建标签字典
labels = y_train.tolist() + y_test.tolist()
labels_types = list(set(labels))
print(labels_types)

labels_dict = {}
labels_index = {"padded_label" : 0}

for index in range(len(labels_types)):
    label = labels_types[index]
    labels_dict.update({label: labels.count(label)})
    labels_index.update({label: index+1})

np.save('y_labels_index.npy', labels_index) 
print(len(labels), len(labels_dict), labels_dict)

[nan, 'AS', 'X', 'DEG', 'M', 'VP', 'NR', 'VV', 'VA', 'ETC', 'LC', 'NT', 'LB', 'DER', 'SB', 'VC', 'IJ', 'AD', 'VE', 'NN', 'DEV', 'CS', 'CD', 'MSP', 'JJ', 'P', 'BA', 'OD', 'FW', 'SP', 'DEC', 'NP', 'PN', 'DT', 'CC', 'PU']
520125 36 {nan: 18426, 'AS': 4118, 'X': 6, 'DEG': 12337, 'M': 13790, 'VP': 1, 'NR': 30570, 'VV': 69858, 'VA': 7755, 'ETC': 1303, 'LC': 7782, 'NT': 9659, 'LB': 245, 'DER': 258, 'SB': 455, 'VC': 5404, 'IJ': 12, 'AD': 36430, 'VE': 3005, 'NN': 136643, 'DEV': 634, 'CS': 892, 'CD': 16182, 'MSP': 1336, 'JJ': 13234, 'P': 17606, 'BA': 755, 'OD': 1675, 'FW': 33, 'SP': 468, 'DEC': 12510, 'NP': 5, 'PN': 6644, 'DT': 5986, 'CC': 7355, 'PU': 76753}


In [9]:
# 构建的Y标签字典
print(labels_index)

{'padded_label': 0, nan: 1, 'AS': 2, 'X': 3, 'DEG': 4, 'M': 5, 'VP': 6, 'NR': 7, 'VV': 8, 'VA': 9, 'ETC': 10, 'LC': 11, 'NT': 12, 'LB': 13, 'DER': 14, 'SB': 15, 'VC': 16, 'IJ': 17, 'AD': 18, 'VE': 19, 'NN': 20, 'DEV': 21, 'CS': 22, 'CD': 23, 'MSP': 24, 'JJ': 25, 'P': 26, 'BA': 27, 'OD': 28, 'FW': 29, 'SP': 30, 'DEC': 31, 'NP': 32, 'PN': 33, 'DT': 34, 'CC': 35, 'PU': 36}


In [10]:
# 按句对X、y进行拆分
def split_corpus_by_sentence(content):
    cleaned_sentence = []
    split_label = content.isnull()
    last_split_index = 0
    index = 0
    while index < len(content):
        current_word = content[index]
        if split_label[index] == True and len(cleaned_sentence) == 0:
            cleaned_sentence.append(np.array(content[last_split_index:index]))
            last_split_index = index + 1
            index += 1
        elif split_label[index] == True  and len(cleaned_sentence) > 0:
            cleaned_sentence.append(np.array(content[last_split_index:index]))
            last_split_index = index + 1
            index += 1
        else:
            index += 1
    return cleaned_sentence

X_train_sent_split = split_corpus_by_sentence(X_train)
y_train_sent_split = split_corpus_by_sentence(y_train)
X_test_sent_split = split_corpus_by_sentence(X_test)
y_test_sent_split = split_corpus_by_sentence(y_test)

print('以句子进行拆分后的句子为：\n', X_train_sent_split[:5])
print('以句子进行拆分后的句子所对应的词性为：\n', y_train_sent_split[:5])

以句子进行拆分后的句子为：
 [array(['上海', '浦东', '开发', '与', '法制', '建设', '同步'], dtype=object), array(['新华社', '上海', '二月', '十日', '电', '（', '记者', '谢金虎', '、', '张持坚', '）'],
      dtype=object), array(['上海', '浦东', '近年', '来', '颁布', '实行', '了', '涉及', '经济', '、', '贸易', '、',
       '建设', '、', '规划', '、', '科技', '、', '文教', '等', '领域', '的', '七十一', '件',
       '法规性', '文件', '，', '确保', '了', '浦东', '开发', '的', '有序', '进行', '。'],
      dtype=object), array(['浦东', '开发', '开放', '是', '一', '项', '振兴', '上海', '，', '建设', '现代化',
       '经济', '、', '贸易', '、', '金融', '中心', '的', '跨世纪', '工程', '，', '因此',
       '大量', '出现', '的', '是', '以前', '不', '曾', '遇到', '过', '的', '新', '情况',
       '、', '新', '问题', '。'], dtype=object), array(['对', '此', '，', '浦东', '不', '是', '简单', '的', '采取', '“', '干', '一', '段',
       '时间', '，', '等', '积累', '了', '经验', '以后', '再', '制定', '法规', '条例', '”',
       '的', '做法', '，', '而', '是', '借鉴', '发达', '国家', '和', '深圳', '等', '特区',
       '的', '经验', '教训', '，', '聘请', '国内外', '有关', '专家', '学者', '，', '积极',
       '、', '及时', '地', '制定', '和', '推

In [11]:
def transfer_label_category_index(origin_labels, labels_types):
    transfered_label = []
    for sentence_labels in origin_labels:
        labels_format_index = [labels_types.index(label) for label in sentence_labels]  # 将标签依据字典转化为序号
        transfered_label.append(labels_format_index)
    return transfered_label

y_train_index = transfer_label_category_index(y_train_sent_split, labels_types)
y_test_index = transfer_label_category_index(y_test_sent_split, labels_types)

print(y_train_index[:5])

[[6, 6, 19, 34, 19, 19, 7], [19, 6, 11, 11, 19, 35, 19, 6, 35, 6, 35], [6, 6, 11, 10, 7, 7, 1, 7, 19, 35, 19, 35, 19, 35, 19, 35, 19, 35, 19, 9, 19, 30, 22, 4, 19, 19, 35, 7, 1, 6, 19, 3, 24, 19, 35], [6, 19, 19, 15, 22, 4, 7, 6, 35, 7, 19, 19, 35, 19, 35, 19, 19, 30, 24, 19, 35, 17, 17, 7, 30, 15, 11, 17, 17, 7, 1, 30, 24, 19, 35, 24, 19, 35], [25, 32, 35, 6, 17, 15, 8, 20, 7, 35, 7, 22, 4, 19, 35, 25, 7, 1, 19, 10, 17, 7, 19, 19, 35, 30, 19, 35, 34, 15, 7, 24, 19, 34, 6, 9, 19, 3, 19, 19, 35, 7, 19, 24, 19, 19, 35, 17, 35, 17, 20, 7, 34, 7, 19, 19, 35, 7, 33, 19, 19, 17, 7, 17, 14, 7, 19, 19, 35]]


In [12]:
MAX_SEQUENCE_LENGTH = 100

# 标签格式转化
# 构建对应（标签样本数，句子长度，标签类别数+1）形状的张量，值全为0
y_train_index_padded = np.zeros((len(y_train_index), MAX_SEQUENCE_LENGTH, len(labels_types)+1), dtype='float', order='C')
y_test_index_padded = np.zeros((len(y_test_index), MAX_SEQUENCE_LENGTH, len(labels_types)+1), dtype='float', order='C')

# 填充张量
for sentence_labels_index in range(len(y_train_index)):
    for label_index in range(len(y_train_index[sentence_labels_index])):
        if label_index < MAX_SEQUENCE_LENGTH:
            y_train_index_padded[sentence_labels_index, label_index, y_train_index[sentence_labels_index][label_index]+1] = 1
    
    if len(y_train_index[sentence_labels_index]) < MAX_SEQUENCE_LENGTH:
        for label_index in range(len(y_train_index[sentence_labels_index]), MAX_SEQUENCE_LENGTH):
            y_train_index_padded[sentence_labels_index, label_index, 0] = 1

# 优化：若为填充的标签，则将其预测为第一位为1

for sentence_labels_index in range(len(y_test_index)):
    for label_index in range(len(y_test_index[sentence_labels_index])):
        if label_index < MAX_SEQUENCE_LENGTH:
            y_test_index_padded[sentence_labels_index, label_index, y_test_index[sentence_labels_index][label_index]+1] = 1
    
    if len(y_test_index[sentence_labels_index]) < MAX_SEQUENCE_LENGTH:
        for label_index in range(len(y_test_index[sentence_labels_index]), MAX_SEQUENCE_LENGTH):
            y_test_index_padded[sentence_labels_index, label_index, 0] = 1

print(y_train_index_padded[:1])

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]


## word2vec模型导入

预训练的word2vec模型采用前人用中文维基百科训练好的模型，请各位同学进入链接下载，并放到相对本脚本 同一级文件夹data 的目录下 并解压。

[word2vec模型链接](https://github.com/Embedding/Chinese-Word-Vectors)

In [13]:
## 1 导入 预训练的词向量
myPath = './data/sgns.wiki.word' # 本地词向量的地址
Word2VecModel = gensim.models.KeyedVectors.load_word2vec_format(myPath) # 读取词向量，以二进制读取

In [14]:
## 2 构造包含所有词语的 list，以及初始化 “词语-序号”字典 和 “词向量”矩阵
vocab_list = [word for word, Vocab in Word2VecModel.wv.vocab.items()]# 存储 所有的 词语

word_index = {" ": 0}# 初始化 `[word : token]` ，后期 tokenize 语料库就是用该词典。
word_vector = {} # 初始化`[word : vector]`字典

# 初始化存储所有向量的大矩阵，留意其中多一位（首行），词向量全为 0，用于 padding补零。
# 行数 为 所有单词数+1 比如 10000+1 ； 列数为 词向量“维度”比如100。
embeddings_matrix = np.zeros((len(vocab_list) + 1, Word2VecModel.vector_size))

C:\Users\admin\AppData\Local\Temp\ipykernel_10168\2531751338.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vocab_list = [word for word, Vocab in Word2VecModel.wv.vocab.items()]# 存储 所有的 词语


In [15]:
## 3 填充 上述 的字典 和 大矩阵
for i in range(len(vocab_list)):
    # print(i)
    word = vocab_list[i]  # 每个词语
    word_index[word] = i + 1 # 词语：序号
    word_vector[word] = Word2VecModel.wv[word] # 词语：词向量
    embeddings_matrix[i + 1] = Word2VecModel.wv[word]  # 词向量矩阵

C:\Users\admin\AppData\Local\Temp\ipykernel_10168\1270982749.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  word_vector[word] = Word2VecModel.wv[word] # 词语：词向量
C:\Users\admin\AppData\Local\Temp\ipykernel_10168\1270982749.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  embeddings_matrix[i + 1] = Word2VecModel.wv[word]  # 词向量矩阵


In [16]:
# ## 2 构造包含所有词语的 list，以及初始化 “词语-序号”字典 和 “词向量”矩阵
# word_index = {" ": 0}# 初始化 `[word : token]` ，后期 tokenize 语料库就是用该词典。
# word_vector = {} # 初始化`[word : vector]`字典

# # 初始化存储所有向量的大矩阵，留意其中多一位（首行），词向量全为 0，用于 padding补零。
# # 行数 为 所有单词数+1 比如 10000+1 ； 列数为 词向量“维度”比如100。
# embeddings_matrix = np.zeros((len(Word2VecModel.index_to_key) + 1, Word2VecModel.vector_size))

# ## 3 填充 上述 的字典 和 大矩阵
# for i in range(len(Word2VecModel.index_to_key)):
#     # print(i)
#     word = Word2VecModel.index_to_key[i]  # 每个词语
#     word_index[word] = i + 1 # 词语：序号
#     word_vector[word] = Word2VecModel[word] # 词语：词向量
#     embeddings_matrix[i + 1] = Word2VecModel[word]  # 词向量矩阵

In [17]:
np.save('x_word_index.npy', word_index)

In [19]:
from keras.preprocessing import sequence
# 序号化 文本，tokenizer句子，并返回每个句子所对应的词语索引

# 由于将词语转化为索引的word_index需要与词向量模型对齐，故在导入词向量模型后再将X进行处理
def tokenizer(texts, word_index):
    data = []
    for sentence in texts:
        new_sentence = []
        for word in sentence:
            try:
                new_sentence.append(word_index[word])  # 把文本中的 词语转化为index
            except:
                new_sentence.append(0)
       
        data.append(new_sentence)
    # 使用kears的内置函数padding对齐句子,好处是输出numpy数组，不用自己转化了
    data = sequence.pad_sequences(data, maxlen = MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
    
    return data

X_train_tokenized = tokenizer(X_train_sent_split, word_index)
X_test_tokenized = tokenizer(X_test_sent_split, word_index)

print(X_train_tokenized[:1])

[[  347 16980   507    10 15537   603  4380     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]]


## 标引网络构建及训练评估

In [20]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
import keras
from keras import optimizers

EMBEDDING_DIM = 300 #词向量维度

model = Sequential()
model.add(Embedding(input_dim = len(embeddings_matrix), # 字典长度
                    output_dim = EMBEDDING_DIM, # 词向量 长度（300）
                    weights=[embeddings_matrix], # 重点：预训练的词向量系数
                    input_length=MAX_SEQUENCE_LENGTH, # 每句话的 最大长度（必须padding） 
                    trainable=False # 是否在 训练的过程中 更新词向量
                   ))
# input shape (Batch_size, Time_step, Input_Sizes)
model.add(LSTM(128, input_shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM), activation='tanh', return_sequences=True))
model.add(Dropout(0.5))
model.add(Dense(64, input_shape=(MAX_SEQUENCE_LENGTH, 128), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(labels_types)+1, input_shape=(MAX_SEQUENCE_LENGTH, 64), activation='softmax'))

adam = optimizers.adam_v2.Adam(lr=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

D:\Anaconda\envs\ml\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          105665400 
_________________________________________________________________
lstm (LSTM)                  (None, 100, 128)          219648    
_________________________________________________________________
dropout (Dropout)            (None, 100, 128)          0         
_________________________________________________________________
dense (Dense)                (None, 100, 64)           8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 64)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 100, 37)           2405      
Total params: 105,895,709
Trainable params: 230,309
Non-trainable params: 105,665,400
____________________________________

In [22]:
print(X_train_tokenized.shape, y_train_index_padded.shape)

(18078, 100) (18078, 100, 37)


In [23]:
model.fit(X_train_tokenized, y_train_index_padded,
          epochs=10,
          batch_size=128,
          verbose=1
         )
print('evaluation!')
score = model.evaluate(X_test_tokenized, y_test_index_padded, batch_size=128)

Epoch 1/10
142/142 [==============================] - 34s 219ms/step - loss: 0.3933 - accuracy: 0.8906
Epoch 2/10
142/142 [==============================] - 31s 221ms/step - loss: 0.1581 - accuracy: 0.9504
Epoch 3/10
142/142 [==============================] - 32s 228ms/step - loss: 0.1356 - accuracy: 0.9578
Epoch 4/10
142/142 [==============================] - 31s 215ms/step - loss: 0.1243 - accuracy: 0.9613
Epoch 5/10
142/142 [==============================] - 31s 216ms/step - loss: 0.1170 - accuracy: 0.9634
Epoch 6/10
142/142 [==============================] - 31s 216ms/step - loss: 0.1118 - accuracy: 0.9649
Epoch 7/10
142/142 [==============================] - 31s 218ms/step - loss: 0.1081 - accuracy: 0.9660
Epoch 8/10
142/142 [==============================] - 31s 217ms/step - loss: 0.1043 - accuracy: 0.9669
Epoch 9/10
142/142 [==============================] - 31s 221ms/step - loss: 0.1016 - accuracy: 0.9680
Epoch 10/10
142/142 [==============================] - 31s 218ms/step - l

In [24]:
model.save('cn_pos_tag.h5')

In [25]:
print('训练的模型经在测试集上验证获得的loss和accuracy为：')
print(score)

训练的模型经在测试集上验证获得的loss和accuracy为：
[0.05551695078611374, 0.9819827675819397]


In [26]:
print(model.predict(X_test_tokenized[:1]))

[[[1.06635525e-05 3.57727771e-07 1.05397652e-04 ... 2.33568018e-03
   6.92393598e-07 3.12632910e-05]
  [6.14345424e-07 1.82051492e-11 1.25356635e-07 ... 8.37220568e-07
   1.73747974e-10 4.92240506e-05]
  [1.05229647e-13 1.94991093e-15 7.08194614e-10 ... 8.24436630e-10
   6.79707114e-07 2.06903022e-10]
  ...
  [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 9.38489382e-15]
  [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 9.38485824e-15]
  [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 9.38482266e-15]]]


In [27]:
print(y_test_index_padded[:1])

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]


In [28]:
print(model.predict(X_test_tokenized[2:3]))

[[[1.2263517e-15 1.9288053e-24 1.4051022e-15 ... 2.3026050e-22
   9.8017287e-13 9.9999928e-01]
  [2.4927356e-06 1.1019267e-08 1.0667166e-05 ... 7.5662494e-05
   3.4589000e-08 3.9010945e-05]
  [3.7808437e-08 7.0922929e-14 3.6413925e-09 ... 2.6348717e-08
   1.0428908e-12 5.3828903e-06]
  ...
  [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
   0.0000000e+00 9.3853953e-15]
  [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
   0.0000000e+00 9.3853233e-15]
  [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
   0.0000000e+00 9.3852165e-15]]]


In [29]:
print(y_test_index_padded[2:3])

[[[0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]


In [30]:
from keras.layers import GRU, SimpleRNN

model_RNN = Sequential()
model_RNN.add(Embedding(input_dim = len(embeddings_matrix), # 字典长度
                    output_dim = EMBEDDING_DIM, # 词向量 长度（300）
                    weights=[embeddings_matrix], # 重点：预训练的词向量系数
                    input_length=MAX_SEQUENCE_LENGTH, # 每句话的 最大长度（必须padding） 
                    trainable=False # 是否在 训练的过程中 更新词向量
                   ))
# input shape (Batch_size, Time_step, Input_Sizes)
model_RNN.add(SimpleRNN(128, input_shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM), activation='tanh', return_sequences=True))
model_RNN.add(Dropout(0.5))
model_RNN.add(Dense(64, input_shape=(MAX_SEQUENCE_LENGTH, 128), activation='relu'))
model_RNN.add(Dropout(0.5))
model_RNN.add(Dense(len(labels_types)+1, input_shape=(MAX_SEQUENCE_LENGTH, 64), activation='softmax'))

adam = optimizers.adam_v2.Adam(lr=0.01, decay=1e-6)
model_RNN.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

model_RNN.fit(X_train_tokenized, y_train_index_padded,
          epochs=2,
          batch_size=128,
          verbose=1
         )

Epoch 1/2
142/142 [==============================] - 17s 112ms/step - loss: 0.7128 - accuracy: 0.8094
Epoch 2/2
142/142 [==============================] - 15s 105ms/step - loss: 0.3378 - accuracy: 0.8940


In [32]:
model_GRU = Sequential()
model_GRU.add(Embedding(input_dim = len(embeddings_matrix), # 字典长度
                    output_dim = EMBEDDING_DIM, # 词向量 长度（300）
                    weights=[embeddings_matrix], # 重点：预训练的词向量系数
                    input_length=MAX_SEQUENCE_LENGTH, # 每句话的 最大长度（必须padding） 
                    trainable=False # 是否在 训练的过程中 更新词向量
                   ))
# input shape (Batch_size, Time_step, Input_Sizes)
model_GRU.add(GRU(128, input_shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM), activation='tanh', return_sequences=True))
model_GRU.add(Dropout(0.5))
model_GRU.add(Dense(64, input_shape=(MAX_SEQUENCE_LENGTH, 128), activation='relu'))
model_GRU.add(Dropout(0.5))
model_GRU.add(Dense(len(labels_types)+1, input_shape=(MAX_SEQUENCE_LENGTH, 64), activation='softmax'))

adam = optimizers.adam_v2.Adam(lr=0.01, decay=1e-6)
model_GRU.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

model_GRU.fit(X_train_tokenized, y_train_index_padded,
          epochs=2,
          batch_size=128,
          verbose=1
         )

Epoch 1/2
142/142 [==============================] - 30s 198ms/step - loss: 0.3547 - accuracy: 0.9104
Epoch 2/2
142/142 [==============================] - 27s 191ms/step - loss: 0.1603 - accuracy: 0.9481


## attention

In [33]:
from keras.utils import np_utils
from keras.layers import *
from keras.models import *
#from keras.optimizers import Adam

EMBEDDING_DIM = 300
lstm_units = 100

# first way attention
def attention_3d_block(inputs):
    #input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Dense(MAX_SEQUENCE_LENGTH, activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    #output_attention_mul = merge([inputs, a_probs], name=’attention_mul’, mode=’mul’)
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul')
    return output_attention_mul

# build RNN model with attention
# inputs = Input(shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM))
inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
embeding1=Embedding(input_dim = len(embeddings_matrix), # 字典长度
                    output_dim = EMBEDDING_DIM, # 词向量 长度（300）
                    weights=[embeddings_matrix], # 重点：预训练的词向量系数
                    input_length=MAX_SEQUENCE_LENGTH, # 每句话的 最大长度（必须padding）
                    trainable=False # 是否在 训练的过程中 更新词向量
                                  )(inputs)
drop1 = Dropout(0.3)(embeding1)
lstm_out = Bidirectional(LSTM(lstm_units, return_sequences=True), name='bilstm')(embeding1)
attention_mul = attention_3d_block(lstm_out)
# attention_flatten = Flatten()(attention_mul)
drop2 = Dropout(0.3)(attention_mul)
output = Dense(len(labels_types)+1, activation='sigmoid')(drop2)
model_att = Model(inputs=inputs, outputs=output)

model_att.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model_att.summary())

print('Training————')
att = model_att.fit(X_train_tokenized, y_train_index_padded, epochs=1, batch_size=128)

print('Testing————–')
loss, accuracy = model_att.evaluate(X_test_tokenized, y_test_index_padded)

print('test loss:', loss)
print('test accuracy:', accuracy)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 100, 300)     105665400   input_1[0][0]                    
__________________________________________________________________________________________________
bilstm (Bidirectional)          (None, 100, 200)     320800      embedding_4[0][0]                
__________________________________________________________________________________________________
permute (Permute)               (None, 200, 100)     0           bilstm[0][0]                     
______________________________________________________________________________________________

In [34]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout,SimpleRNN,GRU,Bidirectional,Input,Flatten,Multiply
import keras
from keras import backend as K


EMBEDDING_DIM = 300 #词向量维度

#两个维度变换过了
input1=Input(shape=(MAX_SEQUENCE_LENGTH,))
embeding1=Embedding(input_dim = len(embeddings_matrix), # 字典长度
                    output_dim = EMBEDDING_DIM, # 词向量 长度（300）
                    weights=[embeddings_matrix], # 重点：预训练的词向量系数
                    input_length=MAX_SEQUENCE_LENGTH, # 每句话的 最大长度（必须padding）
                    trainable=False # 是否在 训练的过程中 更新词向量
                                  )(input1)
LSTM1=LSTM(128, input_shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM), return_sequences=True)(embeding1)
dense1=Dense(128, input_shape=(MAX_SEQUENCE_LENGTH,128), activation='softmax')(LSTM1)
dense2=Dense(128, input_shape=(MAX_SEQUENCE_LENGTH,128), activation='relu')(LSTM1)
attention=Multiply()([dense1,dense2])
dense3=Dense(64, activation='relu')(attention)
dropout2=Dropout(0.5)(dense3)
out=Dense(len(labels_types)+1, activation='softmax')(dropout2)
model2=keras.models.Model(inputs=input1,outputs=out)

adam=keras.optimizers.adam_v2.Adam(lr=0.01,decay=0.0001)
model2.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

print(model2.summary())

model2.fit(X_train_tokenized, y_train_index_padded,
          epochs=2,
          batch_size=128,
          verbose=1
         )
print('evaluation!')
score1 = model2.evaluate(X_test_tokenized, y_test_index_padded, batch_size=128)
score2 = model2.evaluate(X_train_tokenized, y_train_index_padded, batch_size=128)
print('训练的模型经在测试集上验证获得的loss和accuracy为：')
print(score1)
print('训练的模型经在训练集集上验证获得的loss和accuracy为：')
print(score2)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 100, 300)     105665400   input_2[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 100, 128)     219648      embedding_5[0][0]                
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 100, 128)     16512       lstm_2[0][0]                     
____________________________________________________________________________________________